In [45]:
import pandas as pd
from pathlib import Path
import os
import re

# Path to the folder
folder_path = r'C:\Users\ELXY\Documents\Codigos\Python\P84_85\MTO\files'
output_path = r'C:\Users\ELXY\Documents\Codigos\Python\P84_85\MTO'

# List all files in the folder
files = os.listdir(folder_path)

# Create empty list to store dataframes
dfs = []

# Read each file
for file in files:
    if file.endswith(('.xlsx', '.xls', '.csv')):  # Add more extensions if needed
        file_path = os.path.join(folder_path, file)
        df = pd.read_excel(file_path, header=None)
        # Ler o valor de uma célula específica (linha 1, coluna 3)
        modulo = df.iloc[0, 2]  # Índices começam em 0
        # Extrair o nome do módulo da string 'modulo'
        match = re.search(r'MTO - MATERIAL LIST - (.+)', modulo)
        modulo_nome = match.group(1).strip() if match else None
        print(f"Processing file: {file} - Módulo: {modulo_nome}")
        # Extrair o nome da lista (ex: I-LI-3010.2S-1354-200-S2N-005)
        match_lista = re.search(r'(I-LI-\d+\.\d+S-\d+-\d+-S\d+N-\d+)', modulo)
        lista_nome = match_lista.group(1).strip() if match_lista else None

        # Read file skipping first 3 rows and using 4th row as header
        df = pd.read_excel(file_path, skiprows=3, header=0)
        df["modulo_nome"] = modulo_nome
        df["lista_nome"] = lista_nome
        dfs.append(df)

# Combine all dataframes
df_mto = pd.concat(dfs, ignore_index=True)

Processing file: I-LI-3010.2S-1354-200-S2N-005_Attachment 1_Rev 0.xlsx - Módulo: FLARE TOWER
Processing file: I-LI-3010.2S-1400-200-S2N-101_Attachment 1_Rev 0.xlsx - Módulo: MAIN DECK
Processing file: I-LI-3010.2S-1411-200-S2N-005_Attachment 1_Rev 0.xlsx - Módulo: M-01
Processing file: I-LI-3010.2S-1412-200-S2N-005_Attachment 1-Rev 0.xlsx - Módulo: M-02
Processing file: I-LI-3010.2S-1414-200-S2N-005_Attachment 1_Rev 0.xlsx - Módulo: M-04
Processing file: I-LI-3010.2S-1415-200-S2N-005_Attachment 1_Rev 0.xlsx - Módulo: M-05
Processing file: I-LI-3010.2S-1415-200-S2N-105_Attachment 1_Rev 0.xlsx - Módulo: M-05B
Processing file: I-LI-3010.2S-1416-200-S2N-005_Attachment 1_Rev 0.xlsx - Módulo: M-06
Processing file: I-LI-3010.2S-1417-200-S2N-005_Attachment 1_Rev 0.xlsx - Módulo: M-07A
Processing file: I-LI-3010.2S-1417-200-S2N-305_Attachment 1_Rev 0.xlsx - Módulo: M-07B
Processing file: I-LI-3010.2S-1419-200-S2N-005_Attachment 1_Rev 0.xlsx - Módulo: M-09
Processing file: I-LI-3010.2S-1421-200-

In [46]:
# Converter a coluna "Dry Weight in kgs" para float (caso necessário)
df_mto["Dry Weight in kgs"] = pd.to_numeric(df_mto["Dry Weight in kgs"], errors="coerce")
# Agrupar e somar os pesos por "modulo_nome" e "Material Category"
df_peso_por_modulo_material = df_mto.groupby(["modulo_nome", "Material Category"])["Dry Weight in kgs"].sum().reset_index()
display(df_peso_por_modulo_material)

,modulo_nome,Material Category,Dry Weight in kgs
0,FLARE TOWER,Carbon Steel,8216.24648
1,FLARE TOWER,Galv. CS,81.18000
2,FLARE TOWER,Low Temp\nCS,39347.54676
3,FLARE TOWER,SS 310H,7431.86272
4,FLARE TOWER,SS 316/316L,105707.95581
...,...,...,...
131,M-16,SS 316/316L,2983.74594
132,MAIN DECK,CS Inconel\nClad.,20143.97902
133,MAIN DECK,Carbon Steel,1373.47921
134,MAIN DECK,Duplex,108.23856


In [47]:
# Remover linhas onde a coluna "HOLD" contém a palavra "HOLD" (ex: "HOLD 1", "HOLD2", etc)
df_mto_w_hold = df_mto[~df_mto["HOLD"].astype(str).str.contains("HOLD", na=False)]
# Agrupar e somar os pesos por "modulo_nome" e "Material Category" sem as linhas "HOLD"
df_peso_por_modulo_material_w_hold = df_mto_w_hold.groupby(["modulo_nome", "Material Category"])["Dry Weight in kgs"].sum().reset_index()
display(df_peso_por_modulo_material_w_hold)

,modulo_nome,Material Category,Dry Weight in kgs
0,FLARE TOWER,Carbon Steel,4281.41708
1,FLARE TOWER,Galv. CS,75.74000
2,M-01,Carbon Steel,5931.90000
3,M-01,Copper-Nickel,573.09000
4,M-01,Duplex,224.43000
...,...,...,...
111,M-16,SS 316/316L,2979.00594
112,MAIN DECK,CS Inconel\nClad.,57.60000
113,MAIN DECK,Carbon Steel,1079.50958
114,MAIN DECK,Duplex,108.23856


In [48]:
# Converter a coluna "Dry Weight in kgs" para float (caso necessário)
df_mto["Dry Weight in kgs"] = pd.to_numeric(df_mto["Dry Weight in kgs"], errors="coerce")
# Agrupar e somar os pesos por "Material Category"
df_peso_por_material = df_mto.groupby(["Material Category"])["Dry Weight in kgs"].sum().reset_index()
display(df_peso_por_material)

,Material Category,Dry Weight in kgs
0,3.5Ni\nInconel Clad,1.660854e+04
1,CS Inconel\nClad.,5.650996e+05
2,Carbon Steel,9.676527e+05
3,Copper-Nickel,6.376027e+04
4,Duplex,3.063667e+05
5,FRP,8.923456e+04
6,Galv. CS,8.118000e+01
7,Inconel,7.354925e+04
8,Low Temp\nCS,4.626011e+05
9,Organic Coat\nCS,1.415733e+05


In [51]:
mto_file = f"mto.xlsx"

# Salvando arquivos P84 e P85:
with pd.ExcelWriter(mto_file) as writer:
    df_mto.to_excel(writer, sheet_name='MTO', index=False)
    df_peso_por_modulo_material.to_excel(writer, sheet_name='Peso_Material_por_Modulo', index=False)
    df_peso_por_modulo_material_w_hold.to_excel(writer, sheet_name='Peso_Material_sem_Hold', index=False)
    df_peso_por_material.to_excel(writer, sheet_name='Peso_por_Material', index=False)
print(f'Arquivo {mto_file} salvo em: ', Path.cwd())

Arquivo mto.xlsx salvo em:  c:\Users\ELXY\Documents\Codigos\Python\P84_85\MTO
